In [1]:
import csv                                      #to write csv files
import json                                     #to load json data and parse
from dateutil.parser import parse               #to parse date time onjects
import re                                       #regular expressions to clean data
import codecs                                   #not used but has some good encoding options which can be explored
import unicodedata                              #not used (code is commented) but has a good way to normalize the data.
import html                                     #escape html data
import string                                   #string manipulation functions
from nltk.sentiment.vader import SentimentIntensityAnalyzer  #for sentiment calulation
from nltk import tokenize                       #tokenizer to tokenize the words. Not used.
from textblob import TextBlob                   #one more package to calculate sentiment. code commented. not used
sid = SentimentIntensityAnalyzer()              #create an instance of sentiment intensity analyzer to be used later.

/usr/lib/python3.4/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [295]:
#list of words with apostrophes, used while cleaning text to replace apostrope words with individual words.
#works for plain string texts. might not work for byte or unicode encoded texts.
global apos_dict
apos_dict = {
"aren't" : "are not",
"can't" : "can not",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'll" : "I will",
"i'm" : "I am",
"i'd" : "I would",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"y'all": "you all",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":"we will",
"didn't": "did not"
}

In [296]:
#list of cities that we are interested in. 
city_list = set(
["acworth"
,"adairsville"
,"aldora"
,"alpharetta"
,"atlanta"
,"auburn"
,"austell"
,"avondale estates"
,"ball ground"
,"barnesville"
,"berkeley lake"
,"bethlehem"
,"bowdon"
,"braswell"
,"bremen"
,"brooks"
,"buchanan"
,"buford"
,"canton"
,"carl"
,"carrollton"
,"cartersville"
,"centralhatchee"
,"chamblee"
,"clarkston"
,"college park"
,"concord"
,"conyers"
,"covington"
,"cumming"
,"dacula"
,"dallas"
,"dawsonville"
,"decatur"
,"doraville"
,"douglasville"
,"duluth"
,"dunwoody"
,"east point"
,"emerson"
,"ephesus"
,"euharlee"
,"fairburn"
,"fayetteville"
,"flovilla"
,"forest park"
,"franklin"
,"gay"
,"georgia"
,"good hope"
,"grantville"
,"grayson"
,"greenville"
,"griffin"
,"hampton"
,"hapeville"
,"haralson"
,"hiram"
,"holly springs"
,"jackson"
,"jasper"
,"jenkinsburg"
,"johns creek"
,"jonesboro"
,"kennesaw"
,"kingston"
,"lake city"
,"lawrenceville"
,"lilburn"
,"lithonia"
,"locust grove"
,"loganville"
,"lone oak"
,"lovejoy"
,"luthersville"
,"madison"
,"manchester"
,"mansfield"
,"marietta"
,"mcdonough"
,"meansville"
,"milner"
,"milton"
,"molena"
,"monroe"
,"monticello"
,"moreland"
,"morrow"
,"mount zion"
,"mountain park"
,"nelson"
,"newborn"
,"newnan"
,"norcross"
,"orchard hill"
,"oxford"
,"palmetto"
,"peachtree city"
,"pine lake"
,"porterdale"
,"powder springs"
,"riverdale"
,"roopville"
,"roswell"
,"sandy springs"
,"senoia"
,"shady dale"
,"sharpsburg"
,"smyrna"
,"snellville"
,"social circle"
,"statham"
,"stockbridge"
,"stone mountain"
,"sugar hill"
,"sunny side"
,"suwanee"
,"talking rock"
,"tallapoosa"
,"temple"
,"turin"
,"tyrone"
,"union city"
,"villa rica"
,"waco"
,"waleska"
,"walnut grove"
,"warm springs"
,"white"
,"whitesburg"
,"williamson"
,"winder"
,"woodbury"
,"woodstock"
,"woolsey"
,"zebulon"
])


#List of states that we are interested in. 
state_list = set(["ga","georgia"])

#cities/states that we want to filter and remove.
#Note: Since we use bounding box to get tweets, we end up getting tweets from neighboring cities and states.
skip_list = set(["north carolina", "florida", "alabama", "carolina del norte", "carolina del norte"])

In [297]:
#Function which reads the tweet text, looks for apostrophe words and replaces them by individual words and sends the text back
def apos_words(text):
    sentence=[]
    for word in text.split():
        try:
            sentence.append(apos_dict[word.lower()])
        except:
            sentence.append(' '.join(word.split("'")))
    return ' '.join(sentence)

In [335]:
#tweet preprocess 
def tweet_text_preprocess(text):
    #Removing unicode(hex) characters. -> #re.sub(r'\\x[0-9A-Fa-f]+',r'', text)
    #remove the "RT @username" from tweet -> #re.sub(r'@\w+',r'',re.sub(r'rt @\w+:',r'',text)
    text=re.sub(r'@\w+',r'',re.sub(r'rt @\w+:',r'',re.sub(r'\\x[0-9A-Fa-f]+',r'', re.sub(r'\\n',r'',text)))) #,r'(?:RT @[\w_:]+) ',r''
    #Splits the apostrophe words in text -> text=apos_words(text)
    #remove the URLs from tweets -> re.sub('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+',r'',text)
    text=re.sub('(?:(?:https?|ftp):\/\/)?[\w/\-?=%]+\.[\w/\-?=%]+',r'',apos_words(text))
    #Remove any white space characters 
    #Unescape html chars in text -> html.unescape(text)
    #Replace ### with spaces -> replace("#"," ")
    text = ' '.join(re.findall("[\S]+", html.unescape(text))).replace("#"," ").replace("\n",'').replace("\t",'').replace("\r",'').replace("\f",'').replace("\v",'')
    #####text=' '.join(re.findall("[^ \t\n\r\f\v]+", text))
    
    #the below code separates hex codes for emojis and makes them into a list foe each tweet. 
    #This can be used later to calculate sentiment
    #text=re.sub(r'\\x.+[0-9]','', text)
    #li = '\\'.join(re.findall(r'x[0-9A-F]+',text,re.I))
    #print("hex codes: ", li)
    
    return ' '.join(text.split())

In [27]:
#stripping white space characters in string text:
text='Yall ever just been around hella girls you aint want \n ......disgusting'
print("original text: ",text)
print("After stripping white spaces", ' '.join(re.findall("[\S]+", text)))

original text:  Yall ever just been around hella girls you aint want 
 ......disgusting
After stripping white spaces Yall ever just been around hella girls you aint want ......disgusting


In [332]:
#Stripping white space characters in byte encoded text:
t=b'Yall ever just been around hella girls you ain\xe2\x80\x99t want......disgusting'
s=str(t)[2:len(str(t))-1]
print("original text: ",s)
cleaned_text=tweet_text_preprocess(s)
print(' '.join(cleaned_text.split()))

original text:  Yall ever just been around hella girls you ain\xe2\x80\x99t want......disgusting
1  Yall ever just been around hella girls you aint want......disgusting
2  Yall ever just been around hella girls you aint want......disgusting
3  Yall ever just been around hella girls you aint want......disgusting
4  Yall ever just been around hella girls you aint want......disgusting
Yall ever just been around hella girls you aint want......disgusting


In [301]:
#if sid.polarity_scores("happy")['pos'] == 1:
#    print("pos")
#
#sid.polarity_scores("bad")['neu']
#sid.polarity_scores("bad")['neg']

In [357]:
#Function which gets the city name based on the place type mentioned in each tweet. Examine a sample json file for clarity.
def get_city_info(place_type, name, full_name):
    skip_record = other_loc = False
    if place_type == "admin":
        city = name.strip()
        state = name.strip()
    elif place_type == "city":
        city = name.strip()
        if full_name.lower().split(",")[-1].strip() == "ga":
            state = "Georgia"
        else:
            state = full_name.split(",")[-1].strip()
    elif place_type == "poi":
        city = name.strip()
        state = "Georgia"
    elif place_type == "neighborhood":
        if place_type.lower() in city_list:
            city = name.strip()
        else:
            city = full_name.split(",")[-1].strip()
            state = "Georgia" #if city in city_list else "NA"
    else:
        print ("A new placetype has been identified: ", place_type)
    
    #setting the necessary flags
    if city.lower() in skip_list:  skip_record = True
    if city.lower() not in city_list: other_loc = True
    
    return(city,state,skip_record,other_loc)

In [368]:
header = ['Data Source', 'ID', 'User_Id', 'Screen_Name', 'User_Name', 'Original Source', 'Language',
           'Time', 'Date', 'Time_Zone', 'Location', 'City', 'State', 'Country', 'Share_Count', 'Favorite_Count',
           'Comment_Count', 'URL', 'Description', 'Headlines', 'Encoded_Text', 'Cleaned_Text', 'Sentiment', 'Pos_count','Neg_count','Hashtags']

#check whether the file is empty: If it is emoty, write headers.
#writing data with location information and without location information into two separate files:
with open('non_loc.csv','w',newline='') as fileio:
    if fileio.seek(0) in (None, "", 0):
        writer = csv.writer(fileio)
        writer.writerows([header])            
    else:
        pass
with open('loc.csv','w',newline='') as fileio:
    if fileio.seek(0) in (None, "", 0):
        writer = csv.writer(fileio)
        writer.writerows([header])            
    else:
        pass

In [369]:
#initiate the required counters: 
error_records=written_loc_records=written_nonloc_records=no_coords=errors=url_error=errors=input_recs=unwanted_location=plc_error=0

#open two files as output and the json tweet file as input:
with open('twitter_data_location_combined.json','r') as inp, open('non_loc.csv','a',newline='',encoding='utf-8') as op1, open('loc.csv','a',newline='',encoding='utf-8') as op2:
    csvWriter1 = csv.writer(op1)
    csvWriter2 = csv.writer(op2)
    
    #Define a translator which is used to remove punctuations. Refer documentation to understand how translator works.
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    
    #process each record in input file:
    for row in inp:
        input_recs+=1
        #Intiate an empty list. This list will be used to write data into csv file.
        tweet_content = []
        json_data = json.loads(row,encoding='utf-8')
        
        #try to get the geo coordinates (if available). Pupolate 'NA' if coordinates are not available
        try:
            coordinates=json.dumps(json_data["geo"]["coordinates"]).strip("[]")
        except:
            coordinates="NA"
            no_coords+=1
        
        #append the tweet data to list to prepare the output record.
        #tweet_content=[]
        tweet_content.append("Twitter")                               #source
        tweet_content.append(json_data["id_str"])                     #Tweet ID
        tweet_content.append(json_data["user"]["id_str"])             #User ID
        tweet_content.append(json_data["user"]["screen_name"])        #User screen name
        tweet_content.append(json_data["user"]["name"])               #User name
        
        try:
            if (len(json_data["entities"]["urls"])>0):                #URL of the content of tweet
                tweet_original_source = json_data["entities"]["urls"][0]["expanded_url"]
        except:                                                       #If URL is not available, use tweet status link
            tweet_original_source = 'https://twitter.com/' + tweet_user_id + '/status/' + tweet_id    
            url_error+=1
        
        tweet_content.append(tweet_original_source)                   #URL source as extracted above
        
        tweet_content.append(json_data["lang"])                       #Tweet Language
        dt = parse(json_data["created_at"])                           #Date and time, parsed by parser from dateutil
        tweet_content.append(str(dt.time()))                          #get time from above datetime object
        tweet_content.append(str(dt.date()))                          #get date from above datetime object
        tweet_content.append("UTC")                                   #UTC as timezone
        tweet_content.append(coordinates)                             #Latitude and Longitude coordinates of the tweet
        
        #process location details
        skip_record = other_loc = False
        city = state = ''
        try:
            city,state,skip_record,other_loc = get_city_info(json_data["place"]["place_type"],json_data["place"]["name"],json_data["place"]["full_name"])
        except:
            plc_error+=1
            skip_record = True
        #if the location information is not of our interest, skip the record. Otherwise, get other information and write the data
        #Improvement: This can be first evaluated and the remaining information can be obtained if city is of our interest.
        if not skip_record:
            tweet_content.append(city)                                    #City from above try except loop
            tweet_content.append(state)                                   #State
            
            tweet_content.append(json_data["place"]["country_code"])      #Country from which tweet was tweeted
            
            tweet_content.append(json_data["retweet_count"])              #share count
            tweet_content.append(json_data["favorite_count"])             #favourite count
            
            tweet_content.append('NA')                                    #comment count -> this is not available
            tweet_content.append(tweet_original_source)                   #URL of the tweet
            
            #tweet_content.append(json_data["user"]["description"])       #User profile description
            try:
                tweet_content.append(json_data["user"]["description"].encode('utf-8'))
            except:
                tweet_content.append('NA')
            tweet_content.append('NA')                                    #Headlines -> not available
            
            #Get the full text of tweet if available. otherwise, use the available text. Refer process document for more info.
            try:
                tweet_content.append(json_data["extended_tweet"]["full_text"].encode('utf-8'))
                tweet_text=str(json_data["extended_tweet"]["full_text"].encode('utf-8'))
            except:
                tweet_content.append(json_data["text"].encode('utf-8'))
                tweet_text=str(json_data["text"].encode('utf-8'))
            
            #tweet is byte encoded. None of the string operations work directly. So, it is converted to string. 
            #leading "b'" is removed. trailing single quote is also removed. 
            tweet_text=tweet_text[2:len(tweet_text)-1]
            
            #preprocess the tweet to remove hex characters
            cleaned_text=tweet_text_preprocess(tweet_text)
            
            #remove punctuations and convert everything to lower.
            #Improvement: Vader package can consider case of the text and provide sentiment. Need not convert to lower.
            #Might as well try keeping the punctuations. 
            cleaned_text= ' '.join(cleaned_text.translate(translator).split()).lower()
            
            #Append the cleaned text to the list.
            tweet_content.append(cleaned_text)                                               #Cleaned tweet text without punctuations
            
            #Calculate sentiment using sid initiated in the beginning for Sentiment Intensity Analyzer. 
            tweet_content.append(round(sid.polarity_scores(cleaned_text)['compound'],1))     #sentiment
            
            #Positive and negative sentiment words. This was initially used for a visualization suggested. This can be removed.
            #Sentiment indicator using nltk
            positive_count=negative_count=0
            for w in cleaned_text.split(" "):
                if sid.polarity_scores(w)['pos'] == 1:
                    positive_count+=1
                elif sid.polarity_scores(w)['neg'] == 1:
                    negative_count+=1
                else:
                    pass                                              
            tweet_content.append(positive_count)
            tweet_content.append(negative_count)
            
            
            #tweet_content.append(round(TextBlob(cleaned_text).sentiment.polarity,2)) #Sentiment indicator using textblob
            #tweet_content.append(unicodedata.normalize('NFC', tweet_text).encode('ascii', 'ignore'))   
            #tweet_content.append(json_data["text"].encode('utf-8'))       #Write the original text with hex codes. Shall be used later to extract sentiment.
            
            #extract hash tags from json file instead of tweet text.
            hashs=[]
            if len(json_data["entities"]["hashtags"])>0:                  #Extract hash tags from tweets if the length of list >0
                for dicts in json_data["entities"]["hashtags"]:
                    hashs.append(dicts["text"].lower())
                tweet_content.append(','.join(hashs))
            
            #writing non location records and location based records to two different files.
            if other_loc:
                csvWriter1.writerow(tweet_content)
                written_nonloc_records+=1
            else:
                csvWriter2.writerow(tweet_content)
                written_loc_records+=1
        else:
            unwanted_location+=1

print("Input records                    : ", input_recs)
print("Written loc records              : ", written_loc_records)
print("Written non-loc records          : ", written_nonloc_records)
print("Skipped unwanted-loc records     : ", unwanted_location)
print("Records without coords           : ", no_coords)
print("Records with error location      : ", plc_error)
print("Tweets without URLs              : ", url_error)
print("Write errors                     : ", error_records)
print("Total written records            : ", written_loc_records+written_nonloc_records)
#print("Skipped total records            : ", errors)




Input records                    :  479671
Written loc records              :  247892
Written non-loc records          :  21123
Skipped unwanted-loc records     :  210656
Records without coords           :  436002
Records with error location      :  2
Tweets without URLs              :  0
Write errors                     :  0
Total written records            :  269015
